# Project Phase 2: Large Vision and Language Models

## Connexion to our index

In [17]:
import pprint as pp
import requests

host = 'localhost'
port = 9200

user = 'admin' # Add your user name here.
password = 'a.Zerty12.3' # Add your user password here. For testing only. Don't store credentials in code. 
index_name = user

In [18]:
import pprint as pp
from opensearchpy import OpenSearch
from opensearchpy import helpers

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_auth = (user, password),
    use_ssl = True,
    verify_certs = False,
)

if client.indices.exists(index_name):

    resp = client.indices.open(index = index_name)
    print(resp)

    print('\n----------------------------------------------------------------------------------- INDEX SETTINGS')
    settings = client.indices.get_settings(index = index_name)
    pp.pprint(settings)

    print('\n----------------------------------------------------------------------------------- INDEX MAPPINGS')
    mappings = client.indices.get_mapping(index = index_name)
    pp.pprint(mappings)

    print('\n----------------------------------------------------------------------------------- INDEX #DOCs')
    print(client.count(index = index_name))
else:
    print("Index does not exist.")


{'acknowledged': True, 'shards_acknowledged': True}

----------------------------------------------------------------------------------- INDEX SETTINGS
{'admin': {'settings': {'index': {'creation_date': '1744838749384',
                                  'knn': 'true',
                                  'number_of_replicas': '0',
                                  'number_of_shards': '4',
                                  'provided_name': 'admin',
                                  'refresh_interval': '-1',
                                  'replication': {'type': 'DOCUMENT'},
                                  'uuid': 'DTG5g1Q0TmGFnC6ZN7pPWA',
                                  'version': {'created': '136347827'}}}}}

----------------------------------------------------------------------------------- INDEX MAPPINGS
{'admin': {'mappings': {'properties': {'caption_bow': {'analyzer': 'standard',
                                                       'type': 'text'},
                           

/home/lina/miniconda3/envs/nlp-cv-ir/lib/python3.9/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/lina/miniconda3/envs/nlp-cv-ir/lib/python3.9/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/lina/miniconda3/envs/nlp-cv-ir/lib/python3.9/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/lina

In [19]:
# Give every documents added to the index
client.indices.refresh(index=index_name)
query = {
    "query": {
        "match_all": {}
    },
    "size": 100  # Adjust the number to see more results if needed
}

response = client.search(
    index=index_name,
    body=query
)

# Print the total number of documents and some of the documents
print(f"Total hits: {response['hits']['total']['value']}")
for hit in response['hits']['hits']:
    print(hit['_source'])  # Print the source of each document

Total hits: 0


/home/lina/miniconda3/envs/nlp-cv-ir/lib/python3.9/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/lina/miniconda3/envs/nlp-cv-ir/lib/python3.9/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


## Cross-Modal Retrieval

### Updating the mapping of our index

In [20]:
#deleting all the documents in our index to create new ones for this phase
response = client.delete_by_query(
    index=index_name,
    body={
        "query": {
            "match_all": {}  # all the documents
        }
    }
)

print('\nDeleting all the documents:')
print(response)


Deleting all the documents:
{'took': 8, 'timed_out': False, 'total': 0, 'deleted': 0, 'batches': 0, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []}


/home/lina/miniconda3/envs/nlp-cv-ir/lib/python3.9/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [21]:
# Give every documents added to the index: just to check if the index is empty
client.indices.refresh(index=index_name)
query = {
    "query": {
        "match_all": {}
    },
    "size": 100  # Adjust the number to see more results if needed
}

response = client.search(
    index=index_name,
    body=query
)

# Print the total number of documents and some of the documents
print(f"Total hits: {response['hits']['total']['value']}")
for hit in response['hits']['hits']:
    print(hit['_source'])  # Print the source of each document

Total hits: 0


/home/lina/miniconda3/envs/nlp-cv-ir/lib/python3.9/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/lina/miniconda3/envs/nlp-cv-ir/lib/python3.9/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [26]:
# New mapping for image_embedding and image_caption_embedding
mapping_body = {
    "properties": {
        "image_embedding": {
            "type": "knn_vector",
            "dimension": 512, # dimension for CLIP image embeddings
            "index": True,
            "similarity": "cosine"
        },
        "image_caption_embedding": {
            "type": "knn_vector",
            "dimension": 512,  # dimension for CLIP text embeddings
            "index": True,
            "similarity": "cosine"
        }
    }
}

# Update the mapping for the index
response = client.indices.put_mapping(
    index=index_name,
    body=mapping_body,
    ignore=400  # Ignore errors if the mapping already exists
)

print("\nUpdate of the mapping of the index:")
print(response)


Update of the mapping of the index:
{'acknowledged': True}


/home/lina/miniconda3/envs/nlp-cv-ir/lib/python3.9/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [27]:
#checking the mapping of the index

client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_auth = (user, password),
    use_ssl = True,
    verify_certs = False,
)

if client.indices.exists(index_name):

    resp = client.indices.open(index = index_name)
    print(resp)

    print('\n----------------------------------------------------------------------------------- INDEX SETTINGS')
    settings = client.indices.get_settings(index = index_name)
    pp.pprint(settings)

    print('\n----------------------------------------------------------------------------------- INDEX MAPPINGS')
    mappings = client.indices.get_mapping(index = index_name)
    pp.pprint(mappings)

    print('\n----------------------------------------------------------------------------------- INDEX #DOCs')
    print(client.count(index = index_name))
else:
    print("Index does not exist.")

{'acknowledged': True, 'shards_acknowledged': True}

----------------------------------------------------------------------------------- INDEX SETTINGS
{'admin': {'settings': {'index': {'creation_date': '1744838749384',
                                  'knn': 'true',
                                  'number_of_replicas': '0',
                                  'number_of_shards': '4',
                                  'provided_name': 'admin',
                                  'refresh_interval': '-1',
                                  'replication': {'type': 'DOCUMENT'},
                                  'uuid': 'DTG5g1Q0TmGFnC6ZN7pPWA',
                                  'version': {'created': '136347827'}}}}}

----------------------------------------------------------------------------------- INDEX MAPPINGS
{'admin': {'mappings': {'properties': {'caption_bow': {'analyzer': 'standard',
                                                       'type': 'text'},
                           

/home/lina/miniconda3/envs/nlp-cv-ir/lib/python3.9/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/lina/miniconda3/envs/nlp-cv-ir/lib/python3.9/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/lina/miniconda3/envs/nlp-cv-ir/lib/python3.9/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/lina

### Parsing through moments' frames

### Image embedding

### Queries